## DS 7337: Natural Language Processing

## Jaclyn Coate
## Homework 7

#### Spring 2021
#### Natural Language Processing w/ Python: Bird, Klein, & Loper

In [1]:
# Environment Prep
import bs4; print( 'bs4 ' + bs4.__version__)
from bs4 import BeautifulSoup, SoupStrainer

import nltk; print( 'nltk ' + nltk.__version__)
from nltk import word_tokenize, pos_tag, RegexpParser;
from nltk.tokenize import sent_tokenize

import re; print('re ' + re.__version__)
import requests; print('requests ' + requests.__version__)

bs4 4.8.2
nltk 3.4.5
re 2.2.1
requests 2.22.0


In [13]:
# Source links for Top Horror Movies in History and store in review_home_url
review_home_urls = {
    'The Exorcist': 'https://www.imdb.com/title/tt0070047/reviews?ref_=tt_ql_3',
    'Paranormal Activity': 'https://www.imdb.com/title/tt1179904/reviews?ref_=tt_ql_3',
    'Hellraiser': 'https://www.imdb.com/title/tt0093177/reviews?ref_=tt_ql_3',
    'A Nightmare on Elm Street': 'https://www.imdb.com/title/tt0087800/reviews?ref_=tt_ql_3',
    'American Psycho' : 'https://www.imdb.com/title/tt0087800/reviews?ref_=tt_ql_3',
    'The Blair Witch Project': 'https://www.imdb.com/title/tt0185937/reviews?ref_=tt_ql_3',
}

In [14]:
#Helper Functions
# This function is to grab text from the URL: grab_text_from_url
def get_text_from_url(url):
    return requests.get(url).text
text = get_text_from_url(review_home_url['The Exorcist'])

# This function points to the links within the HTML
def get_all_html_links(html):
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer('a', href=True))
    urls = [str(tag.attrs['href']) for tag in tags]
    return urls
all_links = get_all_html_links(text)

# This defintion sets the URL template and allows the unique urls to be grabbed after the root url
def get_review_links(links):
    url_template = 'https://www.imdb.com{}'
   
    return [url_template.format(link) for link in links]

urls = get_review_links(all_links);

# This defines the relevent part of the link by calling our review section to point to HTML post the root 'review'
def relevent_link(link):
    if '/review/' in link:
        return True
    return False
def get_relevent_links(links):
    relevent_links = filter(relevent_link, all_links)
    unique_relevent_links = set(relevent_links)
    return list(unique_relevent_links)
relevent_urls = get_relevent_links(urls)
len(relevent_urls)

# Grabbing reviews from sites
def strain_content(name, attrs):
    if name == 'div' and dict(attrs).get('class', None) == 'content':
        return True
    return False
def clean_review_text(text):
    return re.split('\\n\\n\s+\d+ out of \d+', text)[0]
def get_review_from_url(url):
    html = get_text_from_url(url)
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer(strain_content))
    review = clean_review_text(tags.text)
    return review

# Grabbing reviews from sites
def get_review_from_site(url):
    reviews = []

    reviews_home_text = get_text_from_url(url)
    all_links = get_all_html_links(reviews_home_text)
    relevent_links = get_relevent_links(all_links)

    review_urls = get_review_urls_from_links(relevent_links)
    for url in review_urls:
        reviews.append(get_review_from_url(url))
        # break
    return reviews

# Locating the review sections from the sites 
def get_reviews_from_all_sites():
    all_reviews = []
    review_titles = review_home_urls.keys()
    for title in review_titles:
        review_home_url = review_home_urls[title]
        all_reviews = all_reviews + get_review_from_site(review_home_url)
    return all_reviews

In [15]:
# Grabbing reviews and placing them in one place
all_reviews = get_reviews_from_all_sites()

In [16]:
all_reviews

["\nMore than thirty years on, The Exorcist remains a very powerful film and was a cinematographic milestone in 1973. Repeated duplication of the genre has, no doubt, 'desensitized' a new generation of movie-watchers, though it remains an unnerving masterpiece. It is not difficult to understand why the film generated such a seismic global impact all those years ago, since it imposed an unprecedented sensory attack on the viewer. Regan's vile physical appearance, combined with her vile language and blasphemous diatribe sent a shock wave around the world. Moreover, many people seemed to believe the claims that the film was based on a true story and could therefore actually happen to them. Electricity consumption must have soared for several months in 1973 as people who had seen the film slept with their lights on! It is still not a film I would feel comfortable watching before going to bed. On another level, I found parts of it profoundly moving and actually cried at the end when Regan w

##### HW 7: Question 1
In Python, select any one of the clustering methods covered in this course. Run it over the collection of reviews, and show at least two different ways of clustering the reviews, e.g., changing k in k-Means clustering or changing where you “cut” in Agnes or Diana.  

In [18]:
from nltk.metrics import *
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import requests
import string

In [19]:
# Making sure stop words will be removed
new_stops = list(stopwords.words('english'))

new_stops = new_stops + ['show', 'episode', 'it', 'get', ' it.', 'it\'s', 'series', 'season', 
                         'can', 'no', 've', 'watch', 'watched', 'much', 'lot', 'thus', 'no', 'need', 'needed', 'should\'ve',
                        'it,', 'can.', 'can,']
stop_words = new_stops

In [20]:
#create vectorizer
vectorizer = TfidfVectorizer(stop_words={'english'})

In [21]:
#Perform tf-idf on reviews
links_vect = vectorizer.fit_transform(all_reviews)

In [22]:
#Look at shape
links_vect.shape

(150, 2214)

In [32]:
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
#https://scikit-learn.org/stable/modules/clustering.html#k-means
#https://matplotlib.org/tutorials/introductory/pyplot.html
#https://github.com/wnayden/MSDS-7337/blob/master/20201116%20Homework%207.ipynb
#################################inertia_ is the sum of squared distance from cluster center#################################

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt #import matplotlib to examine intracluster distances

#Cluster
kmeans = KMeans(n_clusters=5, random_state=11)
clustered = kmeans.fit(links_vect)

In [33]:
#Examine 5 Clusters
#https://github.com/wnayden/MSDS-7337/blob/master/20201116%20Homework%207.ipynb
kmeans = KMeans(n_clusters=5, random_state=11)
clustered = kmeans.fit(links_vect)

In [34]:
clustered.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 0, 4, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2,
       0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2,
       2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2,
       3, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 3, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [35]:
clustered.labels_.shape

(150,)

In [36]:
#https://pythonprogramminglanguage.com/kmeans-text-clustering/
#https://github.com/wnayden/MSDS-7337/blob/master/20201116%20Homework%207.ipynb
#Print top words of each cluster
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
words = vectorizer.get_feature_names()
for i in range(5):
    print("cluster:", i)
    for j in order_centroids[i, :10]:
        print(words[j])

cluster: 0
sure
the
top
it
godfather
classics
millennials
rate
50
perfection
cluster: 1
the
and
film
is
films
of
are
to
could
not
cluster: 2
the
and
of
to
is
it
in
that
this
for
cluster: 3
shockingly
grossness
aspects
authentic
hundreds
horrors
karres
spot
wise
judge
cluster: 4
blu
rates
favourite
revisited
recently
45
sets
great
still
films


In [37]:
#Examine 7 Clusters
#https://github.com/wnayden/MSDS-7337/blob/master/20201116%20Homework%207.ipynb
kmeans = KMeans(n_clusters=7, random_state=11)
clustered = kmeans.fit(links_vect)
clustered.labels_
#https://pythonprogramminglanguage.com/kmeans-text-clustering/

#Print top words of each cluster
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
words = vectorizer.get_feature_names()
for i in range(7):
    print("cluster:", i)
    for j in order_centroids[i, :10]:
        print(words[j])

cluster: 0
the
and
you
is
of
it
that
present
to
shocking
cluster: 1
sure
the
top
it
godfather
classics
millennials
rate
50
perfection
cluster: 2
the
gory
was
and
gruesome
genuine
minute
movie
story
maybe
cluster: 3
the
and
of
to
is
it
in
that
for
this
cluster: 4
shockingly
grossness
aspects
authentic
hundreds
horrors
karres
spot
wise
judge
cluster: 5
the
lighting
editing
used
overall
effects
classic
content
sounds
film
cluster: 6
blu
rates
favourite
revisited
recently
45
sets
great
still
films


##### HW 7: Question 2

Try to write a short phrase to characterize (give a natural interpretation of) what each cluster is generally centered on semantically. Is this hard to do in some cases? If so, make note of that fact. 

Cluster 4 seemed to be surrouding those extreme adjectives that you would commonly find in a movie review about horror movies.

* Cluster 4: Shockingly authentic horrors. 

Cluster 5 seems to be focused around the environment or what the stage might be like for those characters. It was rather simple to write a sentence insertin 'are' as a stop word that could/would have been removed. 

* Cluster 5: The lighteing effects are classic. 

##### HW 7: Question 3

Explain which of the two clustering results from question 1 is preferable (if one of them is), and why.

The cluster of 5 seemed to be a better job of dilineating between types of words. I say this because in the 7 clusters the words tended to start getting split apart that were still very similar. For instance you see your strong adjectives start to be separated with the 7 cluster effort. 